In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
from helper_functions import calibration_curves
sns.set_theme(
    context="talk",
    style="ticks",
    rc={"font.sans-serif": "Arial"},
    # font_scale=1.,
)

In [ ]:
# train_set = "mnist"
train_set = "mura"

# eval_datasets = ["mnist", "svhn"]
eval_datasets = ["mura"]

results_folder = f"../../results/{train_set}"
figure_folder = "../../reports/figures"
table_folder = "../../reports/tables"

type_dict = {
    "radial": {"color": "#4E79A7", "style": "solid"},
    "radial_200": {"color": "#4E79A7", "style": "dotted"},
    "radial_200_lowlr": {"color": "#4E79A7", "style": "dashed"},
    "meanfield": {"color": "#F28E2B", "style": "dotted"},
    "laplace": {"color": "#59A14F", "style": "dashed"},
    "lowrank": {"color": "#B6992D", "style": "dashdot"},
    "ml": {"color": "#499894", "style": "solid"},
    "map": {"color": "#E15759", "style": "solid"},
    "ensemble_5": {"color": "#BAB0AC", "style": "solid"},
    "ensemble_10": {"color": "#79706E", "style": "solid"},
    "multiswag_5": {"color": "#D37295", "style": "solid"},
    "multiswag_10": {"color": "#FABFD2", "style": "solid"},
}

types = [
    "ensemble_5",
    "ensemble_10",
    "multiswag_5",
    "multiswag_10",
    "radial",
    # "radial_200",
    # "radial_200_lowlr",
    "meanfield",
    "lowrank",
    "laplace",
    "ml",
    "map",
]

results = pd.DataFrame()
for dataset in eval_datasets:

    df = []
    for type in types:
        with open(f"{results_folder}/{type}.pkl", "rb") as f:
            data = pickle.load(f)[f"eval_{dataset}"]
            df.append(data)
    df = pd.DataFrame.from_dict(df)
    df.insert(0, "Type", types)
    results = pd.concat([results, df])

results = results.rename(columns={
    "Average confidence": "Avg. Conf.",
    "Average confidence when wrong": "Avg. Conf. -",
    "Average confidence when right": "Avg. Conf. +",
})

bins = 10
for i, dataset in enumerate(eval_datasets):
    for type in types:
        criteria = f"`Evaluated on` == '{dataset}' and Type == '{type}'"
        data = results.query(criteria)

        targets = data["Test targets"].values[0][:,None]
        probs = data["Test probabilities"].values[0]
        confidences = np.max(probs, axis=1)

        ece, prob_true, prob_pred, n_in_bins = calibration_curves(targets, probs, bins=bins)
        error = 2*np.sqrt( (prob_true * (1-prob_true)) / n_in_bins[n_in_bins > 0] )
        results.loc[results.eval(criteria), 'ECE'] = ece

# Ensembles

In [ ]:
bins = 10

types = [
    "ensemble_5",
    "ensemble_10",
]

fig, ax = plt.subplots(ncols=len(eval_datasets), nrows=2, sharex="all", sharey="all", figsize=(4*len(eval_datasets), 6))
# fig, ax = plt.subplots(ncols=len(eval_datasets), nrows=2, sharex="all", sharey="all", figsize=(5*len(eval_datasets), 8))
for i, dataset in enumerate(eval_datasets):
    ax_curve = ax[0, i] if len(eval_datasets) > 1 else ax[0]
    ax_hist = ax[1, i] if len(eval_datasets) > 1 else ax[1]
    for type in types:
        criteria = f"`Evaluated on` == '{dataset}' and Type == '{type}'"
        data = results.query(criteria)

        targets = data["Test targets"].values[0][:,None]
        probs = data["Test probabilities"].values[0]
        confidences = np.max(probs, axis=1)

        ece, prob_true, prob_pred, n_in_bins = calibration_curves(targets, probs, bins=bins)
        error = 2*np.sqrt( (prob_true * (1-prob_true)) / n_in_bins[n_in_bins > 0] )
        results.loc[results.eval(criteria), 'ECE'] = ece

        line, = ax_curve.plot(prob_pred, prob_true, label=f"{type}", c=type_dict[type]["color"], linestyle=type_dict[type]["style"])
        ax_curve.errorbar(x=prob_pred, y=prob_true, yerr=error, c=line.get_color(), linestyle=type_dict[type]["style"])

        ax_hist.bar(np.linspace(0, 1, bins+1)[-bins:]-1/(2*bins), n_in_bins / np.sum(n_in_bins), width=1/bins, color=line.get_color(), alpha=0.5, label=f"{type}")

    ls = np.linspace(0, 1)
    ax_curve.plot(ls, ls, "--", color="grey", alpha=0.3)
    ax_curve.set(
        xlim=(0, 1),
        ylim=(0, 1),
        title=dataset.upper(),
    )
    ax_hist.set(
        xlim=(0, 1),
        ylim=(0, 1),
    )
    if i == 0:
        ax_curve.set(
            ylabel="True probability",
        )
        ax_hist.set(
            xlabel="Predicted probability",
            ylabel="Frequency",
        )
    if i == len(eval_datasets)-1:
        ax_hist.legend(*ax_curve.get_legend_handles_labels(), frameon=False, fontsize="small")
        ax_hist.set(
            xlabel="Predicted probability",
        )

fig.tight_layout()
sns.despine(fig)

fig.savefig(f"{figure_folder}/{train_set.capitalize()}Ensembles.pdf")

results.query("`Evaluated on` in @eval_datasets and `Type` in @types").drop(["Test targets", "Test probabilities"], axis=1)

# Guides

In [ ]:
bins = 10

types = [
    # "ml",
    # "map",
    "laplace",
    "meanfield",
    "radial",
    # "radial_200",
    # "radial_200_lowlr",
    "lowrank",
]


fig, ax = plt.subplots(ncols=len(eval_datasets), nrows=2, sharex="all", sharey="all", figsize=(4*len(eval_datasets),6))
for i, dataset in enumerate(eval_datasets):
    ax_curve = ax[0, i] if len(eval_datasets) > 1 else ax[0]
    ax_hist = ax[1, i] if len(eval_datasets) > 1 else ax[1]
    for type in types:
        criteria = f"`Evaluated on` == '{dataset}' and Type == '{type}'"
        data = results.query(criteria)

        targets = data["Test targets"].values[0][:,None]
        probs = data["Test probabilities"].values[0]
        confidences = np.max(probs, axis=1)

        ece, prob_true, prob_pred, n_in_bins = calibration_curves(targets, probs, bins=bins)
        error = 2*np.sqrt( (prob_true * (1-prob_true)) / n_in_bins[n_in_bins > 0] )
        results.loc[results.eval(criteria), 'ECE'] = ece

        line, = ax_curve.plot(prob_pred, prob_true, label=f"{type}", c=type_dict[type]["color"], linestyle=type_dict[type]["style"])
        ax_curve.errorbar(x=prob_pred, y=prob_true, yerr=error, c=line.get_color(), linestyle=type_dict[type]["style"])

        ax_hist.bar(np.linspace(0, 1, bins+1)[-bins:]-1/(2*bins), n_in_bins / np.sum(n_in_bins), width=1/bins, color=line.get_color(), alpha=0.5, label=f"{type}")

    ls = np.linspace(0, 1)
    ax_curve.plot(ls, ls, "--", color="grey", alpha=0.3)
    ax_curve.set(
        xlim=(0, 1),
        ylim=(0, 1),
        title=dataset.upper(),
    )
    ax_hist.set(
        xlim=(0, 1),
        ylim=(0, 1),
    )
    if i == 0:
        ax_curve.set(
            ylabel="True probability",
        )
        ax_hist.set(
            xlabel="Predicted probability",
            ylabel="Frequency",
        )
    if i == len(eval_datasets)-1:
        ax_hist.legend(*ax_curve.get_legend_handles_labels(), frameon=False, fontsize="small")
        ax_hist.set(
            xlabel="Predicted probability",
        )

fig.tight_layout()
sns.despine(fig)

fig.savefig(f"{figure_folder}/{train_set.capitalize()}VI.pdf")

results.query("`Evaluated on` in @eval_datasets and `Type` in @types").drop(["Test targets", "Test probabilities"], axis=1)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(14,4))
for guide in [
    "meanfield",
    "lowrank",
    "radial",
    # "radial_200",
    # "radial_200_lowlr",
]:
    with open(f"{results_folder}/{guide}.pkl", "rb") as f:
        data = pickle.load(f)
        epochs = np.arange(len(data["Training ELBO"]))
        ax[0].plot(epochs, data["Training ELBO"].squeeze(), label=guide, c=type_dict[guide]["color"], linestyle=type_dict[guide]["style"])
        ax[1].plot(epochs, data["Validation log-likelihood"].squeeze(), label=guide, c=type_dict[guide]["color"], linestyle=type_dict[guide]["style"])
        ax[2].plot(epochs, data["Validation accuracy"].squeeze(), label=guide, c=type_dict[guide]["color"], linestyle=type_dict[guide]["style"])
ax[0].set(
    xlabel="Epoch",
    ylabel="ELBO loss",
)
ax[1].set(
    xlabel="Epoch",
    ylabel="Log-likelihood",
)
ax[2].set(
    xlabel="Epoch",
    ylabel="Accuracy",
)
ax[0].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax[0].legend(frameon=False, fontsize="x-small")

fig.tight_layout()
sns.despine(fig)

# MultiSWAG

In [ ]:
bins = 10

types = [
    "multiswag_5",
    "multiswag_10",
]

fig, ax = plt.subplots(ncols=len(eval_datasets), nrows=2, sharex="all", sharey="all", figsize=(4*len(eval_datasets),6))
for i, dataset in enumerate(eval_datasets):
    ax_curve = ax[0, i] if len(eval_datasets) > 1 else ax[0]
    ax_hist = ax[1, i] if len(eval_datasets) > 1 else ax[1]
    for type in types:
        criteria = f"`Evaluated on` == '{dataset}' and Type == '{type}'"
        data = results.query(criteria)

        targets = data["Test targets"].values[0][:,None]
        probs = data["Test probabilities"].values[0]
        confidences = np.max(probs, axis=1)

        ece, prob_true, prob_pred, n_in_bins = calibration_curves(targets, probs, bins=bins)
        error = 2*np.sqrt( (prob_true * (1-prob_true)) / n_in_bins[n_in_bins > 0] )
        results.loc[results.eval(criteria), 'ECE'] = ece

        line, = ax_curve.plot(prob_pred, prob_true, label=f"{type}", c=type_dict[type]["color"], linestyle=type_dict[type]["style"])
        ax_curve.errorbar(x=prob_pred, y=prob_true, yerr=error, c=line.get_color(), linestyle=type_dict[type]["style"])

        ax_hist.bar(np.linspace(0, 1, bins+1)[-bins:]-1/(2*bins), n_in_bins / np.sum(n_in_bins), width=1/bins, color=line.get_color(), alpha=0.5, label=f"{type}")

    ls = np.linspace(0, 1)
    ax_curve.plot(ls, ls, "--", color="grey", alpha=0.3)
    ax_curve.set(
        xlim=(0, 1),
        ylim=(0, 1),
        title=dataset.upper(),
    )
    ax_hist.set(
        xlim=(0, 1),
        ylim=(0, 1),
    )
    if i == 0:
        ax_curve.set(
            ylabel="True probability",
        )
        ax_hist.set(
            xlabel="Predicted probability",
            ylabel="Frequency",
        )
    if i == len(eval_datasets)-1:
        ax_hist.legend(*ax_curve.get_legend_handles_labels(), frameon=False, fontsize="small")
        ax_hist.set(
            xlabel="Predicted probability",
        )

fig.tight_layout()
sns.despine(fig)

fig.savefig(f"{figure_folder}/{train_set.capitalize()}MultiSwag.pdf")

results.query("`Evaluated on` in @eval_datasets and `Type` in @types").drop(["Test targets", "Test probabilities"], axis=1)

In [ ]:
results.drop(["Test targets", "Test probabilities"], axis=1)

In [ ]:
latex_table = results\
    .drop(["Test targets", "Test probabilities"], axis=1)\
    .replace({
        "mnist": "MNIST",
        "svhn": "SVHN",
        "mura": "MURA",
        "ensemble_5": "Ensemble@5",
        "ensemble_10": "Ensemble@10",
        "multiswag_5": "MultiSWAG@5",
        "multiswag_10": "MultiSWAG@10",
        "radial": "Radial",
        "meanfield": "Mean-field",
        "lowrank": "Low-rank",
        "laplace": "Laplace",
        "ml": "ML",
        "map": "MAP",
    })\
    .to_latex(index=False, float_format="%.4f")
print(latex_table)
with open(f"{table_folder}/{train_set}_full.tex", "w") as f:
    f.write(latex_table)